In [ ]:
import tabula
import pandas as pd
import re
import unicodedata
from pathlib import Path

sequenciaDeTabelas = ["10-11","12-13","14-28","29-38","39-55","56-71","72-85"]



LocalNT = Path().resolve()
caminhoPDF = LocalNT.parent /"data" / "raw" / "eOuve - Limeira.pdf"
tabelas = []

dirData = LocalNT.parent / "data"
dirProcessado = dirData / "processed"
dirProcessado.mkdir(exist_ok=True)


modeloSituacao = ['aberto',	'agendado',	'cancelado',	'concluido'	,'em_andamento',	'pendente',	'total']
modeloTipo = ['denuncia',	'doacao',	'elogio',	'informacao',	'reclamacao',	'simplifique'	,'solicitacao',	'sugestao',	'total']

def limparTexto(tabela,coluna):
    

    def limpezaDeCelula(texto):
        if (pd.isna(texto)):
            return texto
        texto =str(texto).replace('\r',' ')

        nfkdForm = unicodedata.normalize('NFKD',texto)
        textoSemAcento = nfkdForm.encode('ASCII','ignore').decode('ASCII')


        textoLimpo = re.sub(r'[^a-zA-Z0-9\s]','',textoSemAcento)
        textoLimpo = re.sub(r'\s+',' ',textoLimpo).strip()

        return textoLimpo

    tabela[coluna] = tabela[coluna].apply(limpezaDeCelula)
    return tabela
    

def limparNomeColuna(nomeColuna):
    nomeColuna = str(nomeColuna)

    nfkdForm = unicodedata.normalize('NFKD',nomeColuna)
    nomeLimpo = nfkdForm.encode('ASCII','ignore').decode('ASCII')

    nomeLimpo = nomeLimpo.lower()

    nomeLimpo = re.sub(r'[-–—\s/.]+','_',nomeLimpo)
    nomeLimpo = re.sub(r'[^a-z0-9_]','',nomeLimpo)
    nomeLimpo = re.sub(r'__','_',nomeLimpo).strip('_')


    return nomeLimpo







def limparValores(tabela,listaDeColunas):
    for coluna in listaDeColunas:
        tabela[coluna] = tabela[coluna].astype(str).str.replace('.','',regex = False).str.replace(',','.',regex = False)
        tabela[coluna] = pd.to_numeric(tabela[coluna], errors='coerce')
        tabela[coluna] = tabela[coluna].fillna(0).astype('Int64')

    return tabela



#para evitar a criacao de outra lista de colunas sem o parametro Total, o metodo abaixo tem a variavel colunasParaSomar que recebe todas as colunas -1 que é a Total
def calcularTotal(tabela,modeloColunas):
    colunasParaSomar = [limparNomeColuna(col) for col in modeloColunas[:-1]]
    tabela['total'] = tabela[colunasParaSomar].sum(axis=1)
    return tabela





for sequencia in sequenciaDeTabelas:

  tabela = tabula.read_pdf(
        caminhoPDF,
        pages=sequencia,
        lattice=True,
        multiple_tables=False,
        encoding = "latin1"
        
        
        )[0];

  tabela.columns = [limparNomeColuna(col) for col in tabela.columns]
  tabela.drop(tabela.tail(1).index,inplace=True)
  tabelas.append(tabela)


tabela1 = tabelas[0]
tabela2 = tabelas[1]
tabela3 = tabelas[2]
tabela4 = tabelas[3]
tabela5 = tabelas[4]
tabela6 = tabelas[5]
tabela7 = tabelas[6]







valoresSituacao = {'Aberto' : int,	'Agendado' : int,	'Cancelado': int,	'Concluído': int	,'Em andamento': int,	'Pendente': int,'Total' : int}
valoresTipo = {'Denúncia': int,	'Doação': int,	'Elogio': int,	'Informação': int,	'Reclamação': int,	'Simplifique': int	,'Solicitação': int,	'Sugestão': int,'Total' : int}







tabela1 = limparValores(tabela1,modeloTipo)
tabela2 = limparValores(tabela2,modeloSituacao)
tabela3 = limparValores(tabela3,modeloTipo)
tabela4 = limparValores(tabela4,modeloSituacao)
tabela5 = limparValores(tabela5,modeloTipo)
tabela6 = limparValores(tabela6,modeloSituacao)
tabela7 = limparValores(tabela7,modeloTipo)


tabela1 = calcularTotal(tabela1,modeloTipo)
tabela2 = calcularTotal(tabela2,modeloSituacao)
tabela3 = calcularTotal(tabela3,modeloTipo)
tabela4 = calcularTotal(tabela4,modeloSituacao)
tabela5 = calcularTotal(tabela5,modeloTipo)
tabela6 = calcularTotal(tabela6,modeloSituacao)
tabela7 = calcularTotal(tabela7,modeloTipo)



tabela1 = limparTexto(tabela1,'secretaria')
tabela2 = limparTexto(tabela2,'secretaria')
tabela3 = limparTexto(tabela3,'assunto')
tabela4 = limparTexto(tabela4,'assunto')
tabela5 = limparTexto(tabela5,'bairro')
tabela6 = limparTexto(tabela6,'bairro')
tabela7 = limparTexto(tabela7,'secretaria')

#o codigo abaixo indica que praticamente todo o conteudo foi lido como String(object)
#display(tabela1.dtypes)
#display(tabela2.dtypes) 
#display(tabela3.dtypes) 
#display(tabela4.dtypes) 
#display(tabela5.dtypes) 
#display(tabela6.dtypes) 
#display(tabela7.dtypes) 


listaDeTabelas = [tabela1,tabela2,tabela3,tabela4,tabela5,tabela6,tabela7]

def tabelasParaCsv(listacomTabelas):
    n = 1
    for tabela in listacomTabelas:
        nomeArquivo = f'Tabela{n}_TratadaHackaton.csv'
        caminho = dirProcessado / nomeArquivo
        tabela.to_csv(caminho,index=False,header=True)
        n = n + 1

tabelasParaCsv(listaDeTabelas)



display(tabela1)
display(tabela2)
display(tabela3)
display(tabela4)
display(tabela5)
display(tabela6)
display(tabela7)




,Secretaria,Denúncia,Doação,Elogio,Informação,Reclamação,Simplifique,Solicitação,Sugestão,Total
0,Animais,0,0,0,0,0,0,0,0,0.000
1,Denúncia - Descarte Irregular,172,0,0,0,0,0,150,0,322.000
2,Direitos Humanos -\rDenúncias de discriminação...,1,0,0,0,1,0,0,0,2.000
3,LGPD - Lei Geral de Proteção\rde Dados,17,0,0,0,0,0,30,0,47.000
4,PAT - Posto de Atendimento\rdo Trabalhador,0,0,0,0,0,0,0,0,0.000
5,Secretaria de Agronegócios e\rÁreas Rurais,2,0,0,0,1,0,326,2,331.000
6,Secretaria de Comunicação,44,0,15,10.267,115,0,24,37,10.502
7,Secretaria de Esportes e\rLazer,0,0,0,0,1,0,3,95,99.000
8,Secretaria de Fazenda,43,0,0,0,10,0,494,3,550.000
9,Secretaria de Meio Ambiente\re Saneamento,139,0,0,0,4,0,468,3,614.000
